In [1]:
# list of supported layers
from tensorflow.keras.layers import (
    Input,
    Activation,
    AveragePooling2D,
    BatchNormalization,
    Conv2D,
    Dense,
    Dropout,
    Flatten,
    GlobalAveragePooling2D,
    GlobalMaxPooling2D,
    MaxPooling2D,
    ReLU,
    Softmax,
    )
from tensorflow.keras import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
# load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
# convert y_train and y_test to one-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [4]:
# reshape X_train and X_test to 4D tensor
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

#normalizing
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.0
X_test /= 255.0

In [5]:
inputs = Input(shape=(28,28,1))
out = Conv2D(4, 3, use_bias=False)(inputs)
out = BatchNormalization()(out)
out = Activation('relu')(out)
out = MaxPooling2D()(out)
out = Conv2D(8, 3, use_bias=True, strides=2)(out)
out = ReLU()(out)
out = AveragePooling2D()(out)
out = Flatten()(out)
# out = Dropout(0.5)(out)
out = Dense(10, activation="softmax")(out)
model = Model(inputs, out)

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 4)         36        
                                                                 
 batch_normalization (BatchN  (None, 26, 26, 4)        16        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 26, 26, 4)         0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 4)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 6, 6, 8)           296   

In [7]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['acc']
    )

In [8]:
model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/15


2023-11-26 21:47:52.776729: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1875/1875 [==============================] - 11s 6ms/step - loss: 0.5203 - acc: 0.8386 - val_loss: 0.2099 - val_acc: 0.9363
Epoch 2/15
1875/1875 [==============================] - 11s 6ms/step - loss: 0.1926 - acc: 0.9419 - val_loss: 0.1497 - val_acc: 0.9543
Epoch 3/15
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1551 - acc: 0.9522 - val_loss: 0.1263 - val_acc: 0.9591
Epoch 4/15
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1361 - acc: 0.9580 - val_loss: 0.1139 - val_acc: 0.9628
Epoch 5/15
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1253 - acc: 0.9617 - val_loss: 0.1031 - val_acc: 0.9679
Epoch 6/15
1875/1875 [==============================] - 11s 6ms/step - loss: 0.1168 - acc: 0.9636 - val_loss: 0.0976 - val_acc: 0.9697
Epoch 7/15
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1113 - acc: 0.9650 - val_loss: 0.0923 - val_acc: 0.9711
Epoch 8/15
1875/1875 [==============================] - 10s 5ms/st

In [9]:
model.save('model.h5')